In [1]:
from anytree import Node, RenderTree, PreOrderIter
import json
from pymongo import MongoClient
import glob
import json

In [2]:
client = MongoClient('mongodb://localhost:27017/')

db = client.get_database('tweets')

In [40]:
COLLECTION_NAME = 'data_false_2'
tweets = db[COLLECTION_NAME].find({})
followers = db['followers'].find({})

tweet_user_ids = []
for tweet in tweets:
    tweet_user_ids.append(tweet['user']['id'])

tweet_user_ids

[57441414,
 22736172,
 557193955,
 3899337568,
 569513105,
 564158458,
 1254146990482395137,
 350165465,
 1301672012016947201,
 1190870943167594497,
 1374039117818904584,
 566479653,
 1222754262188482560,
 1622170904,
 178665364,
 4867635922,
 1229029861,
 1372967825120268298,
 841413934703902720,
 1258471016537325568,
 20422072,
 1303054664427081730,
 890832677548556288,
 1339710588008747009,
 1267538645486952453,
 4106923993,
 1563299498,
 888092403139198976,
 294824682,
 230691455,
 833013356,
 4458557717,
 443376427,
 3158134002,
 1490635454,
 961896485674725376,
 1291259699313115139,
 1154250308106641408,
 4017099076,
 9267212,
 218474991,
 20555838,
 739176138073907200,
 3078672341,
 586322289,
 1313774340215840769,
 146027825,
 1378351121631043587,
 1270453030824132608,
 1489476924,
 900325259764785153,
 48360202,
 15770253,
 2668527302,
 62935309,
 1925176123,
 418227801,
 877395414248378369,
 1130553464197488641,
 43827403,
 1380661593533341698,
 968708824600403969,
 891423165

In [4]:
# get time difference of parent tweet and child tweet
from datetime import datetime
def difference_datetime(child_datetime, parent_datetime):
    first_datetime = datetime.strftime(datetime.strptime(child_datetime,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
    first_datetime = datetime.strptime(first_datetime, '%Y-%m-%d %H:%M:%S')
    later_datetime = datetime.strftime(datetime.strptime(parent_datetime,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
    later_datetime = datetime.strptime(later_datetime, '%Y-%m-%d %H:%M:%S')
    return first_datetime - later_datetime

In [5]:
def add_depth_nodes(root, depth_n, tweet_user_ids):
    # Add depth n users
    #get the length of the node
    a = [node.path for node in PreOrderIter(root)]
#     print(a)
    names = []
    for i in range(len(a)):
        depth_of_node = len(a[i])
        if depth_of_node == depth_n:
            names.append(a[i][depth_of_node-1])
    print(names[:5])
          
    # Add depth_n nodes to the cascade
    for child in names:
        followers = db['followers'].find({'user_id': child.name})
        follower = list(followers)
        if not len(follower) == 0:
            child_followers = follower[0]['followers'] 
            child_potential_user_ids = list(set(child_followers) & set(tweet_user_ids))
            # add depth 2 users to the tree and tweet created time
            for user_id in child_potential_user_ids:
                tweets = db[COLLECTION_NAME].find({'user.id':{'$in': child_potential_user_ids}})
                for tweet in tweets:
                    if tweet['user']['id'] == user_id:
                        difference = difference_datetime(tweet['created_at'], child.created_at).total_seconds()
                        if difference >= 0:
                            globals()[f'user_{user_id}'] = Node(name = user_id, created_at = tweet['created_at'], difference_seconds = difference , parent = child)

    for pre, fill, node in RenderTree(root): 
        print("%s user_id: %s, diff_s: %s" % (pre, node.name, node.difference_seconds))
    return root

In [6]:
# remove redundant nodes
# if there are redundant user_ids keep only the most loweres diff_s
# find duplicates
import collections
import anytree
from anytree import Node, RenderTree, AsciiStyle, PreOrderIter



def remove_duplicates(root) :
    #get the length of the node
    a = [node.path for node in PreOrderIter(root)]
#     print(a)
    names = []
    for i in range(len(a)):
        depth_of_node = len(a[i])
        names.append(a[i][depth_of_node-1].name)
    duplicates = [item for item, count in collections.Counter(names).items() if count > 1]
    print(len(duplicates))
    
    # remove duplicates    
    all_nodes_tobe_remove = []
    for duplicate in duplicates:
    #     print(duplicate)
        nodes = anytree.search.findall_by_attr(root, duplicate)
    #     print(nodes)
        node_created_at = set()
        node_different_seconds = []
        # if not crated_at is equal
            # keep the olderst node
        # else
            # keep the closest node to the parent

        # get closest node to the parent
        difference_seconds_sorted = sorted(nodes, key=lambda x: x.difference_seconds)

        for node in nodes:
            node_created_at.add(node.created_at)
            node_different_seconds.append(node.difference_seconds)

        if not len(node_created_at) ==  1:
            print('keep the olderst node', difference_seconds_sorted[0])
            all_nodes_tobe_remove.extend(difference_seconds_sorted[1:])
        else:
    #         print('keep the closest node to the parent: ', min(node_different_seconds), difference_seconds_sorted[0])
            all_nodes_tobe_remove.extend(difference_seconds_sorted[1:])
    print(len(all_nodes_tobe_remove))
    for node in all_nodes_tobe_remove:
        node.parent = None

In [7]:
def multiple_root_users(root_tweet):
    for root_user in root_tweet:
        tweets = db[COLLECTION_NAME].find({})
        followers = db['followers'].find({})
        for tweet in tweets:
            try:
                user_id = root_user
                created_at = tweet['retweeted_status']['created_at']
            except KeyError as e:
                continue    
#         print('User id:',user_id, created_at)
        root = Node(name = user_id, created_at = created_at, difference_seconds =  0)

#         print(root.name)

        follower = db['followers'].find_one({'user_id' : root.name})
        try:
            print(follower['user_id'], len(follower['followers']))
        except:
            print('None, DB does not have the user: ', root.name)
            continue

In [41]:
'''
has a root user 1
root user has followers 1
If a root user followers intersects with tweets, they are cascade depth one users 1
if a cascade depth one user followers intersects with tweets, they are cascade depth two users
...
if a cascade depth n-1 user followers intersects with tweets, they are cascade depth n users

Lets say parent user publish a tweet and the child re-tweet that exact tweet.
child timestamp must be after the parent's time stamp
If child has many parents, parent with the most recent time stamp will be the owner of the child tweet  
'''
tweets = db[COLLECTION_NAME].find({})
#get root tweet
root_tweet = set()
for tweet in tweets:
    try:
        root_tweet.add(tweet['retweeted_status']['user']['id'])
    except KeyError as e:
        continue
print(root_tweet)

tweets = db[COLLECTION_NAME].find({})

# Create root node user_id, created_at

if len(root_tweet) == 1:
    for tweet in tweets:
        try:
            user_id = tweet['retweeted_status']['user']['id']
            created_at = tweet['retweeted_status']['created_at']
        except KeyError as e:
            continue
            
    print(user_id, created_at)

    root = Node(name = user_id, created_at = created_at, difference_seconds =  0)
    print(root.name)

    follower = db['followers'].find_one({'user_id' : root.name})
    try:
        print(follower['user_id'])
    except:
        print('None, DB does not have the user: ', root.name)
        
    #get depth 1 users

    if follower['user_id'] == root.name:
        root_followers = follower['followers']           

    root_followers

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    tweet_user_ids = []
    for tweet in tweets:
        tweet_user_ids.append(tweet['user']['id'])

    tweet_user_ids

    cascade_depth_one_user_ids = list(set(root_followers) & set(tweet_user_ids))

    len(cascade_depth_one_user_ids)

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    # add depth 1 users to the tree
    for user_id in cascade_depth_one_user_ids:
        tweets = db[COLLECTION_NAME].find({})
        for tweet in tweets:
            if tweet['user']['id'] == user_id:
                print(tweet['created_at'])
                difference = difference_datetime(tweet['created_at'], root.created_at).total_seconds()
                print(difference)
                globals()[f'user_{user_id}'] = Node(name = user_id, created_at = tweet['created_at'], difference_seconds = difference , parent = root)


    for pre, fill, node in RenderTree(root):
        print("%s user_id: %s, created_at: %s, difference_seconds: %s" % (pre, node.name, node.created_at, node.difference_seconds))
else:
    multiple_root_users(root_tweet)
#     raise('Cascade has more than one root user')

{57441414}
57441414 Wed May 12 06:53:36 +0000 2021
57441414
57441414


KeyboardInterrupt: 

In [14]:
len(root_tweet)

9

In [ ]:
5695632,874488674, 16893929, 196724330, 15752235, 966317909659103232, 16012783, 5695632, 403765458, 4011928402, 16351765, 16666454, 36876990, 18927441, 25754433, 84111298, 457984599, 61558438, 519897737, 2300835530, 32913997, 882445459, 44202038, 474464413, 89005882, 335206333, 344778430, 1391686780798210052, 16640775, 287603080, 5392522, 18728203, 701725963, 318680721, 19201304, 21110041, 1138852762463154177, 847811211752243200, 14838173, 2261025822, 22705952, 18082466, 1245158946, 24736682, 900085327863218176, 428333, 1268569059731353600, 145692463, 23674801, 1381303367071305729, 17525171, 1153391118773604352, 25817278, 767609351037214720, 255801799, 126051146, 15791186, 18908628, 1359293410847383558, 780460754910732288, 14221917, 444054117, 2269308902, 14697575, 20562924, 57443437, 2828568811, 35773039, 241972592, 429112432, 931039604, 1339526904, 23447418, 4170491, 20087934, 14330880, 20402945, 3191437439, 1147804652039069696, 1329524222109097995, 14500743, 1282297722364727296, 9633802, 1052626822645596161, 25589776, 17710740, 215863060, 226024854, 1333137558893719552, 41085467, 624413, 1143770884974399488, 1295043616742805506, 295860511, 353701419, 1233104313771184129, 768552674719567876, 15108530, 17525171, 48310197, 1278251649803251713, 17852343, 19898168, 133663801, 20583993, 158865339, 472122299, 1652541, 18774524, 980924206132195328, 849800022094880768, 1371529262566608908, 63485764, 356631237, 1283802294186848257, 149516743, 96877126, 21694413, 19407053, 14946895, 16396368, 1276154115773599744, 7313362, 2987385810, 1232689061049126912, 596792665, 786324746719924224, 1260307716385443840, 1310782718603456513, 2735591, 488554346, 26329195, 23922797, 2291106157, 1303630426234408961, 15969142, 19013879, 1016487483138138112, 145802745, 17121146, 1291779641519169536, 14790524, 14594813, 1381246086363365376, 620136960, 24402433, 1370932230, 15169032, 1089607242565984258, 1294649677372153856, 163384337, 1352378437445570567, 79238164, 16973333, 1277745817386975232, 2339238427, 36008477, 809558047, 921033470712918016, 260828194, 374256165, 14301229, 988851384605913088, 2375800382, 19498559, 15438913, 809229705073147904, 17077313, 835941444, 12480582, 1190493069977632768, 21545542, 212145230, 1040650542840782848, 7744592, 989813532786077696, 3702234197, 457984599, 428649561, 747043567936352261, 14296157, 1220432811875880967, 527516258, 2778438762, 1250158141997752332, 189870199, 17602680, 18195067, 27959419, 14229123, 14173315, 224657541, 380648579, 21015687, 224374922, 940290547767496705, 2248538251, 1570225806, 1211718190255861760, 1344804182465916930, 44728980, 63944344, 9763482, 3012907162, 925728652880482305, 755114148, 3713954474, 572517552, 262168753, 14897840, 1238446299650723845, 1279908907004067842, 1126322371, 1338342474280996866, 1124066353739128832, 3315404490, 2940904657, 980540524737150976, 940801984025845761, 1374874868068548617, 1173297035367329798, 27987687, 21208813, 25508589, 19976943, 1077824277041299456, 931279170826412033, 35203319, 43901688, 1265560405314854914, 513627393, 1166384476081070081, 15792387, 16640775, 1280606167144660992, 2873884427, 3331875599, 14692117, 3156281110, 887816658877677568, 19242778, 779515880333910016, 1314655890138705924, 21748010, 758269741, 1278610637271203840, 251935030, 1305909315493130242, 14294848, 245615938, 2461887810, 50318149, 94706501, 111556423, 1332970519424102407, 1168472395, 829882414763753472, 17365337, 733005584946110465, 1195328245618491392, 1226723708871426048, 1001913176756248576, 1197684524630585345, 15969142, 1138576987948244992, 15771003, 389477245, 17908607, 1205508480, 23879043, 534079877, 3044516229, 1000132999, 2784919443, 19032473, 331700122, 1372207883903975426, 142099869, 1231516588077199360, 15872418, 3863925669, 38142380, 55615408, 138705329, 816238800569507842, 997638434394066944, 816477408391929856, 2474805697, 136514502, 16743886, 996670904523423744, 16688083, 713475201284046849, 380285402, 11928542, 9627102, 1063387106918174720, 1206794218213126144, 179903977, 1066727437533351938, 16331756, 64643056, 109975024, 1248561401196675072  

In [11]:
for i in range(2, 10):
    root = add_depth_nodes(root, i, tweet_user_ids)
    remove_duplicates(root)

NameError: name 'root' is not defined